In [ ]:
import numpy as np
from random import shuffle
import os
from scipy.fftpack import fft
from sklearn import preprocessing  # 0-1编码
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [ ]:
def max_min(train_x, test_x):
    # x = (x - Min) / (Max - Min)
    scalar = preprocessing.MinMaxScaler().fit(train_x)
    train_x = scalar.transform(train_x)
    test_x = scalar.transform(test_x)
    return train_x, test_x

# https://blog.csdn.net/qq_27825451/article/details/88553441
def fft_normalize(x):

    fft_x = fft(x)  # 快速傅里叶变换
    abs_x = np.abs(fft_x)  # 取复数的绝对值，即复数的模(双边频谱)
    N = len(x)
    abs_half_x = abs_x[range(int(N / 2))]
    normalization_x = abs_x / N  # 归一化处理（双边频谱）
    normalization_half_x = normalization_x[range(int(N / 2))]  # 由于对称性，只取一半区间（单边频谱）

    # plt.figure()
    # plt.plot(half_number, normalization_half_x, 'blue')
    # plt.title('单边振幅谱(归一化)', fontsize=9, color='blue')
    # plt.show()

    return normalization_half_x

# https://blog.csdn.net/sinat_24259567/article/details/93889547
def add_noise(signal, SNR=None):
    if SNR == None:
        return signal
    else:
        noise = np.random.randn(signal.shape[0], signal.shape[1], signal.shape[2])   #产生N(0,1)噪声数据
        # noise = np.random.randn(signal.shape[0])

        noise = noise - np.mean(noise)  #均值为0
        signal_power = np.linalg.norm(signal - signal.mean()) ** 2 / signal.size  #此处是信号的std**2
        # linalg=linear（线性）+algebra（代数），norm则表示范数。默认求二范数
        noise_variance = signal_power / np.power(10, (SNR / 10))    # np.power(x,y)计算x的y次方  #此处是噪声的std**2
        noise = (np.sqrt(noise_variance) / np.std(noise)) * noise
        signal_noise = noise + signal

        # Ps = (np.linalg.norm(signal - signal.mean())) ** 2  # signal power
        # Pn = (np.linalg.norm(signal - signal_noise)) ** 2  # noise power
        # snr_ = 10 * np.log10(Ps / Pn)
        # print(snr_)

        return signal_noise


'''
所有样本都通过快速傅立叶变换（FFT）转换为频谱。为了规范化数据，通过最大-最小映射将频谱样本缩放到0到1的范围
'''
def cut_samples(SNR=None):
    '''
    每种状况得到400个样本
    '''
    results = np.zeros(shape=(10, 400, 600))
    temporary_s = np.zeros(shape=(400, 1200))
    fft_temporary_s = np.zeros(shape=(400, 600))

    '''
    下载cwru数据集
    '''
    files = os.listdir('C:\\Users\\LENOVO\\Desktop\\论文\\数据集\\cwru12k驱动端\\12k_txt_hebing')
    files.sort()  # 对列表排序
    for i in range(10):
        domain = os.path.abspath(
            r'C:\\Users\\LENOVO\\Desktop\\论文\\数据集\\cwru12k驱动端\\12k_txt_hebing')  # 获取文件夹的路径，此处其实没必要这么写，目的是为了熟悉os的文件夹操作
        file = os.path.join(domain, files[i])
        s = np.loadtxt(file)

        for x in range(400):
            temporary_s[x] = s[1000*x:1200+1000*x]
            fft_temporary_s[x] = fft_normalize(temporary_s[x])
            # if i==1:
            # #     # if x==150 or x==350 :
            # #     print(files[i])
            # #     plt.plot(temporary_s[x])
            # #     plt.show()
            #
            #     plt.plot(fft_temporary_s[x])
            #     plt.show()

            #     noise = np.random.randn(fft_temporary_s[x].shape[0])
            #
            #     noise = noise - np.mean(noise)  # 均值为0
            #     signal_power = np.linalg.norm(fft_temporary_s[x] - fft_temporary_s[x].mean()) ** 2 / fft_temporary_s[x].size  # 此处是信号的std**2
            #     #
            #     noise_variance = signal_power / np.power(10, (SNR / 10))
            #     noise = (np.sqrt(noise_variance) / np.std(noise)) * noise
            #     plt.plot(noise, c='r')
            #     plt.show()
        results[i] = fft_temporary_s

    results = add_noise(results, SNR)

    return results




# 划分训练集和测试集
def make_datasets(results):
    '''输入10*400*1200的原始样本'''
    train_x = np.zeros(shape=(10, 320, 600))
    train_y = np.zeros(shape=(10, 320, 1))
    test_x = np.zeros(shape=(10, 80, 600))
    test_y = np.zeros(shape=(10, 80, 1))

    for i in range(10):
        s = results[i]
        # 打乱顺序
        index_s = [a for a in range(len(s))]
        shuffle(index_s)
        s = s[index_s]
        # 对每种类型都划分训练集和测试集
        train_x[i] = s[:320]
        test_x[i] = s[320:400]

        # 填写标签
        train_y[i, :] = i
        test_y[i, :] = i

    #将十种类型的训练集和测试集分别合并并打乱
    x1 = train_x[0]
    y1 = train_y[0]
    x2 = test_x[0]
    y2 = test_y[0]
    for i in range(9):
        x1 = np.row_stack((x1, train_x[i + 1]))
        x2 = np.row_stack((x2, test_x[i + 1]))

        y1 = np.row_stack((y1, train_y[i + 1]))
        y2 = np.row_stack((y2, test_y[i + 1]))


    index_x1 = [i for i in range(len(x1))]
    index_x2 = [i for i in range(len(x2))]

    shuffle(index_x1)
    shuffle(index_x2)

    x1 = x1[index_x1]
    y1 = y1[index_x1]
    x2 = x2[index_x2]
    y2 = y2[index_x2]

    x1, x2 = max_min(x1, x2)
    y1 = y1.astype(np.int64)
    y2 = y2.astype(np.int64)


    return x1, y1, x2, y2   #分别代表：训练集样本，训练集标签，测试集样本，测试集标签


In [ ]:
import tensorflow as tf
import numpy as np
import itertools
import time
import os
import random
from sklearn.metrics import *

In [ ]:
class DHA_DDA():
    def __init__(self, unit1, unit2, unit3, model_name='dha_dda', train_step=10, fault_num=10,
                 batch_size=500, lr=0.8, R_constant = 1.0, P=1.0, P_R2Cm=1.0, P_class=0.0001):
        """
        :param DataFram:  数据集
        :param train_step:训练步数
        :param R_constant:初始化半径
        :param lr:        优化器学习率
        :param P:         每个球体的惩罚系数
        :param P_R2Cm:    球体之间的惩罚系数
        :param P_class:   加速收敛系数
        """
        self.train_step = train_step
        self.lr = lr
        self.P = P
        self.P_R2Cm = P_R2Cm
        self.P_class = P_class
        self.R_constant = R_constant

        # 编码器部分
        self.lambda1 = 1e-4  # 权重衰减项
        self.noise_factor = 0.05
        self.keep_prop = 0.9  # 数据保留的比例， 用来防止过拟合
        self.unit1 = unit1
        self.unit2 = unit2
        self.unit3 = unit3
        self.batch_size = batch_size
        self.pop = np.array([600, self.unit1, self.unit2, self.unit3, 1])
        self.model_name = model_name
        self.dha_dda = tf.Graph()
        self.fault_num = fault_num

        self.data_space = {}
        self.index_space = {}
        self.activations = []
        self.names = {}
        self.split_num = []  # 每个类别的数据的个数

        # lrp
        self.weights = []
        self.activations = []


    # 编码器的损失函数
    def encoder_loss(self, data, outputdata, hidden_w, output_w):

        error_loss = tf.reduce_mean(tf.square(outputdata - data))
        tf.add_to_collection("losses", error_loss)
        # 参数L2正则化
        regularizer = tf.contrib.layers.l2_regularizer(self.lambda1)
        regularization = regularizer(hidden_w) + regularizer(output_w)
        tf.add_to_collection("losses", regularization)
        # get_collection函数获取指定集合中的所有个体，这里是获取所有损失值，并在 add_n() 函数中进行加和运算
        loss_dae = tf.add_n(tf.get_collection("losses"))

        optimizer_dae = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss_dae)
        return loss_dae, optimizer_dae


    # 在圆心外惩罚
    def g1n_term(self, var, center, Rm):
        g1n = tf.linalg.norm(var - center, axis=1) - Rm  # 点到球心的距离 - 半径 = 点到球的距离
        g1n_max = tf.clip_by_value(g1n, 0, 1e10)
        penalty = tf.reduce_mean(g1n_max)  # if res>0, penalty = res else penalty = 0
        return penalty

    # 在圆心内惩罚
    def g2n_term(self, var, center, Rm):
        g2n = Rm - tf.linalg.norm(var - center, axis=1)  # 半径 - 点到球心的距离 = 点到球边缘的距离
        g2n_max = tf.clip_by_value(g2n, 0, 1e10)
        penalty = tf.reduce_mean(g2n_max)  # if res>0,penalty = res else penalty = 0
        return penalty


    # DHA分类器的损失函数
    def dha_loss(self):
        with self.dha_dda.as_default():
            with tf.name_scope(self.model_name):

                with tf.name_scope('loss_pow'):
                    split_list = np.arange(0, len(self.split_num))
                    for i in range(len(self.split_num)):
                        with tf.name_scope('loss_pow{}'.format(i)):
                            split_list_ = np.delete(split_list, i)
                            if len(split_list_) == 1:
                                U_ = self.names['U{}'.format(split_list_[0])]
                            else:
                                U_ = tf.concat([self.names['U{}'.format(j)] for j in split_list_], 0)

                            g1n1 = self.g1n_term(self.names['U{}'.format(i)],  # layer：weight * v + bias
                                                 self.names['Cm{}'.format(i)],  # 球心
                                                 self.names['R{}'.format(i)])  # 半径
                            g2n1 = self.g2n_term(U_,
                                                 self.names['Cm{}'.format(i)],
                                                 self.names['R{}'.format(i)])

                            Rn = tf.where(tf.greater(np.float64(0), self.names['R{}'.format(i)]),
                                          self.names['R{}'.format(i)], 0)
                            self.names['loss{}_pow'.format(i)] = tf.pow(g1n1, 2) + tf.pow(g2n1, 2) + tf.pow(Rn, 2)

                    loss_pow = 0
                    for i in range(len(self.split_num)):
                        loss_pow = loss_pow + self.names['loss{}_pow'.format(i)]


                with tf.name_scope('lossR2Cm'):
                    combine = list(itertools.combinations(np.arange(0, len(self.split_num)).tolist(), 2))
                    loss_R2Cm = 0
                    for i in combine:
                        with tf.name_scope('lossR2Cm{}{}'.format(i[0], i[1])):
                            Cm_normal = (self.names['R{}'.format(i[0])] + self.names['R{}'.format(i[1])]) \
                                        - tf.linalg.norm(self.names['Cm{}'.format(i[0])] - self.names['Cm{}'.format(i[1])])
                            loss_R2Cm = loss_R2Cm + tf.where(tf.greater(Cm_normal, 0), Cm_normal, 0)


                with tf.name_scope('loss_class'):
                    loss_class = 0.0
                    for i in range(len(self.split_num)):
                        loss_class = loss_class + tf.linalg.norm(self.names['U{}'.format(i)] - self.names['Cm{}'.format(i)])


                with tf.name_scope('loss_all'):
                    self.loss_all = self.P * (loss_pow) + self.P_R2Cm * loss_R2Cm + self.P_class * loss_class

            return self.loss_all


    def gen_model(self, data_noisy, train_label):
        with self.dha_dda.as_default():
            with tf.name_scope(self.model_name):

                for class_i in range(self.fault_num):
                    i_d = np.argwhere(train_label == class_i)
                    i_d = i_d[:,0]
                    self.split_num.append(len(i_d))
                    self.data_space['data{}'.format(class_i)] = data_noisy[i_d]
                    self.index_space['index{}'.format(class_i)] = i_d  # 二维


                with tf.name_scope('InitVariable'):
                    # 动态变量名设置
                    for i in range(len(self.split_num)):
                        with tf.name_scope('V{}'.format(i)):
                            self.names['V{}'.format(i)] = tf.placeholder(
                                tf.float64, shape=[None, data_noisy.shape[1]], name='Input{}'.format(i))


                    with tf.name_scope('weight'):
                        self.hidden_w1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[0], self.pop[1]]),
                                                        trainable=True), tf.float64, name='hidden_w1')  # float32
                        output_w1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[1], self.pop[0]]),
                                                        trainable=True), tf.float64, name='output_w1')
                        self.hidden_w2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[1], self.pop[2]]),
                                                        trainable=True), tf.float64, name='hidden_w2')  # float32
                        output_w2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[2], self.pop[1]]),
                                                        trainable=True), tf.float64, name='output_w2')
                        self.hidden_w3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[2], self.pop[3]]),
                                                        trainable=True), tf.float64, name='hidden_w3')  # float32
                        out_w3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[3], self.pop[2]]),
                                                     trainable=True), tf.float64, name='out_w3')

                        self.weights = [self.hidden_w1, self.hidden_w2, self.hidden_w3]


                    with tf.name_scope('bias'):
                        self.hidden_b1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[1]]),
                                                        trainable=True), tf.float64, name='hidden_b1')
                        output_b1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[0]]),
                                                        trainable=True), tf.float64, name='output_b1')
                        self.hidden_b2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[2]]),
                                                        trainable=True), tf.float64, name='hidden_b2')
                        output_b2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[1]]),
                                                        trainable=True), tf.float64, name='output_b2')
                        self.hidden_b3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[3]]),
                                                        trainable=True), tf.float64, name='hidden_b3')
                        out_b3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[2]]),
                                                     trainable=True), tf.float64, name='out_b3')


                    # R_constant:初始化半径
                    with tf.name_scope('R'):
                        for i in range(len(self.split_num)):
                            self.names['R{}'.format(i)] = tf.Variable(initial_value=self.R_constant,
                                                                 dtype=tf.float64, name='R{}'.format(i), trainable=True)


                with tf.name_scope('encoder_layer'):
                    for i in range(len(self.split_num)):

                        hidden1 = tf.nn.sigmoid(tf.add(tf.matmul(self.names['V{}'.format(i)], self.hidden_w1), self.hidden_b1))
                        output_ae1 = tf.nn.sigmoid(tf.add(tf.matmul(hidden1, output_w1), output_b1))
                        hidden2 = tf.nn.sigmoid(tf.add(tf.matmul(hidden1, self.hidden_w2), self.hidden_b2))
                        output_ae2 = tf.nn.sigmoid(tf.add(tf.matmul(hidden2, output_w2), output_b2))
                        self.names['U{}'.format(i)] = tf.nn.sigmoid(tf.add(tf.matmul(hidden2, self.hidden_w3), self.hidden_b3))
                        out_ae3 = tf.nn.sigmoid(tf.add(tf.matmul(self.names['U{}'.format(i)], out_w3), out_b3))

                        # 逐层训练
                        self.loss_ae1, self.optimizer_ae1 = self.encoder_loss(
                            self.names['V{}'.format(i)], output_ae1, self.hidden_w1,  output_w1)

                        self.loss_ae2, self.optimizer_ae2 = self.encoder_loss(
                            hidden1, output_ae2, self.hidden_w2, output_w2)

                        self.loss_ae3, self.optimizer_ae3 = self.encoder_loss(
                            hidden2, out_ae3, self.hidden_w3, out_w3)

                    self.activations = [self.names['V{}'.format(i)], hidden1, hidden2, self.names['U{}'.format(i)]]


                # 球心
                with tf.name_scope('circle'):
                    for i in range(len(self.split_num)):
                        self.names['Cm{}'.format(i)] = tf.reduce_mean(self.names['U{}'.format(i)], axis=0)


    def fit(self, train_data, train_label, tree_i):
        '''
        :param train_data: 训练数据集
        :param train_label: 训练集的标签，输入的标签应该是一位的，即1,2, 3......
        :return:
        '''
        with self.dha_dda.as_default():
            with tf.name_scope(self.model_name):

                data_noisy = train_data + self.noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train_data.shape)
                self.gen_model(data_noisy, train_label)
                dha_loss = self.dha_loss()


                with tf.name_scope('Optimizer'):
                    learning_rate = tf.train.exponential_decay(self.lr, self.train_step, decay_steps=100,
                                                               decay_rate=0.8)
                    train_op = tf.train.AdamOptimizer(learning_rate).minimize(dha_loss)


                for weight in self.weights:
                    tf.add_to_collection('LRP_DDA_weights'+str(tree_i), weight)

                for act in self.activations:
                    tf.add_to_collection('LRP_DDA_activations'+str(tree_i), act)

                saver = tf.train.Saver()
                with tf.Session(graph=self.dha_dda) as sess:
                    sess.run(tf.global_variables_initializer())
                    # writer = tf.summary.FileWriter("demo_class", sess.graph)
                    # writer.close()

                    feed_dicts = {}
                    for i in range(len(self.split_num)):
                        feed_dicts[self.names['V{}'.format(i)]] = self.data_space['data{}'.format(i)]

                    sess.run([self.loss_ae1, self.optimizer_ae1, self.loss_ae2, self.optimizer_ae2,
                              self.loss_ae3, self.optimizer_ae3], feed_dict=feed_dicts)

                    self.loss_list = []
                    self.R_history = []

                    print('Enter train the Space........')
                    t1_ = time.time()
                    feed_batch_dicts = {}

                    for j in range(self.train_step):
                        for batch_i in range(int(np.ceil(len(train_data) / self.batch_size))):
                            for i in range(len(self.split_num)):
                                feed_batch_dicts[self.names['V{}'.format(i)]] = random.choices(
                                    self.data_space['data{}'.format(i)], k=int(self.batch_size/self.fault_num))
                            _ = sess.run(train_op, feed_dict=feed_batch_dicts)
                            loss = sess.run(self.loss_all, feed_dict=feed_batch_dicts)
                        self.loss_list.append(loss)
                        R = sess.run([self.names['R{}'.format(i)] for i in range(len(self.split_num))])  # 半径
                        self.R_history.append(R)
                        print(" Epoch", j, ": loss : ", loss)

                        # if loss < 1.0:
                        #     break

                    t2_ = time.time()
                    print('训练时间：%.2f s'% (t2_ - t1_))


                    self.R_list = []
                    self.circle = []
                    self.circle = sess.run([self.names['Cm{}'.format(i)] for i in range(self.fault_num)], feed_dict=feed_dicts)
                    self.R_list = sess.run([self.names['R{}'.format(i)] for i in range(self.fault_num)])


                    w1, w2, w3 = sess.run([self.hidden_w1, self.hidden_w2, self.hidden_w3])
                    b1, b2, b3 = sess.run([self.hidden_b1, self.hidden_b2, self.hidden_b3])
                    self.encoder_weight = [w1, w2, w3]
                    self.encoder_bias = [b1, b2, b3]


                    u_list = sess.run([self.names['U{}'.format(i)] for i in range(self.fault_num)], feed_dict=feed_dicts)
                    DHA_pre = np.zeros(shape=[train_label.shape[0], 1])
                    for i in range(len(self.split_num)): # 每个类的数据
                        var = u_list[i]
                        dis = np.zeros(shape=[var.shape[0], len(self.split_num)])
                        for j in range(len(self.split_num)):  # 和每个球心求距离

                            center = tf.reshape(tf.tile(self.circle[j], [self.split_num[i]]),
                                                [self.split_num[i], len(self.circle[j])])   # tf.tile 对张量进行复制
                            dis_c = sess.run(tf.linalg.norm(var - center, axis=1))
                            dis_c = dis_c / self.R_list[j]
                            dis[:, j] = dis_c

                        tmp_id = self.index_space['index{}'.format(i)]
                        DHA_pre[tmp_id] = np.argmin(dis, axis=1).reshape(-1, 1)

                    DHA_pre = DHA_pre.astype(np.int64)
                    acc = tf.reduce_mean(tf.cast(tf.equal(DHA_pre, train_label), tf.float32))
                    acc = sess.run(acc)

                    # 画r和loss的变化曲线图
                    dir1 = './DHA_DDA_jnu/'
                    if not os.path.exists(dir1):
                        os.mkdir(dir1)
                    # picturesdir = './DHA_DDA/pictures/'
                    # if not os.path.exists(picturesdir):
                    #     os.mkdir(picturesdir)
                    # fig1 = Visualization(train_label, DHA_pre, picturesdir)
                    # fig1.plot_R(self.R_history, self.split_num)
                    # fig1.plot_loss(self.loss_list)

                    saver.save(sess, "./DHA_DDA_jnu/model_" + str(tree_i))  # 用于LRP
                    sess.close()
                    return DHA_pre, self.circle, self.R_list, self.encoder_weight, self.encoder_bias, loss, acc



    def test(self, testdata, test_label, Cm, R, w, b, test=False):
        with tf.Session() as sess:
            # data_noisy = testdata + self.noise_factor * np.random.normal(loc=0.0, scale=1.0, size=testdata.shape)
            hidden1 = tf.nn.sigmoid(tf.add(tf.matmul(testdata, w[0]), b[0]))
            hidden2 = tf.nn.sigmoid(tf.add(tf.matmul(hidden1, w[1]), b[1]))
            space_test = tf.nn.sigmoid(tf.add(tf.matmul(hidden2, w[2]), b[2]))

            m = space_test.shape[0]
            test_pre = np.zeros(shape=[testdata.shape[0],  self.fault_num])
            for i in range(len(self.split_num)):

                center = tf.reshape(tf.tile(Cm[i], [m]), [m, len(Cm[i])])  # tf.tile 对张量进行复制
                distance = sess.run(tf.linalg.norm(space_test - center, axis=1))
                distance /= R[i]
                test_pre[:, i] = distance

            test_pre = np.argmin(test_pre, axis=1).astype(np.int64).reshape(-1, 1)
            acc = tf.reduce_mean(tf.cast(tf.equal(test_pre, test_label), tf.float32))
            acc = sess.run(acc)
            print(acc)

            picturesdir = './DHA_DDA_jnu/pictures/'
            if not os.path.exists(picturesdir):
                os.mkdir(picturesdir)

            acc = accuracy_score(test_label, test_pre)
            print('accuracy：', acc)
            rec_w = recall_score(test_label, test_pre, average='macro')
            print('recall score (macro)：', rec_w)
            f1_w = f1_score(test_label, test_pre, average='macro')
            print('f1_score (macro)：', f1_w)
            precision_w = precision_score(test_label, test_pre, average='macro')
            print('precision_score (macro)：', precision_w)

            if test == True:
                fig2 = Visualization(test_label, test_pre, picturesdir)
                acc = fig2.plot_confusion_matrix('DHA_DDA test HAR Confusion Matrix')  # 画混淆矩阵
                space_test = sess.run(space_test)
                fig2.pca_2D(space_test)
                fig2.tsne_3d(space_test)
                return acc, test_pre, space_test

            # fig2 = Visualization(test_label, test_pre, picturesdir)
            # acc = fig2.plot_confusion_matrix('DHA_DDA test HAR Confusion Matrix')  # 画混淆矩阵

            # fig2.plot_roc('roc')
            # sess.close()
            else:
                return acc, test_pre

In [ ]:
class DHA_CDA():
    def __init__(self, unit1, unit2, unit3, model_name='dha_cda', train_step=10, fault_num=10,
                 batch_size=500, lr=0.8, R_constant=1.0, P=1.0, P_R2Cm=1.0, P_class=0.0001):
        """
        :param DataFram:  数据集
        :param train_step:训练步数
        :param R_constant:初始化半径
        :param lr:        优化器学习率
        :param P:         每个球体的惩罚系数
        :param P_R2Cm:    球体之间的惩罚系数
        :param P_class:   加速收敛系数
        """
        self.train_step = train_step
        self.lr = lr
        self.P = P
        self.P_R2Cm = P_R2Cm
        self.P_class = P_class
        self.R_constant = R_constant


        # 编码器部分
        self.delta1 = 1e-3  # 雅克比惩罚项系数

        self.unit1 = unit1
        self.unit2 = unit2
        self.unit3 = unit3
        self.batch_size = batch_size
        self.pop = np.array([600, self.unit1, self.unit2, self.unit3, 1])
        self.model_name = model_name
        self.dha_cda = tf.Graph()
        self.fault_num = fault_num

        self.data_space = {}
        self.index_space = {}
        self.names = {}
        self.split_num = []  # 每个类别的数据的个数

        # lrp
        self.weights = []
        self.activations = []


    # 编码器的损失函数
    def encoder_loss(self, data, outputdata, hidden_w, hidden):

        error_loss = tf.reduce_mean(tf.square(outputdata - data))
        # 雅克比矩阵 https://wiseodd.github.io/techblog/2016/12/05/contractive-autoencoder/
        J_w = tf.transpose(hidden_w)
        d_sigmoid = hidden * (1 - hidden)
        contractive = self.delta1 * tf.reduce_sum(d_sigmoid ** 2 * tf.reduce_sum(J_w ** 2, axis=1), axis=1)

        loss_ae = error_loss + contractive
        optimizer_ae = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_ae, var_list=tf.trainable_variables())

        return loss_ae, optimizer_ae


    # 在圆心外惩罚
    # tf.clip_by_value(A, min, max)：输入一个张量A，把A中的每一个元素的值都压缩在min和max之间。
    # 小于min的让它等于min，大于max的元素的值等于max。
    def g1n_term(self, var, center, Rm):
        g1n = tf.linalg.norm(var - center, axis=1) - Rm  # 点到球心的距离 - 半径 = 点到球的距离
        g1n_max = tf.clip_by_value(g1n, 0, 1e10)
        penalty = tf.reduce_mean(g1n_max)  # if res>0, penalty = res else penalty = 0
        return penalty

    # 在圆心内惩罚
    def g2n_term(self, var, center, Rm):
        g2n = Rm - tf.linalg.norm(var - center, axis=1)  # 半径 - 点到球心的距离 = 点到球边缘的距离
        g2n_max = tf.clip_by_value(g2n, 0, 1e10)
        penalty = tf.reduce_mean(g2n_max)  # if res>0,penalty = res else penalty = 0
        return penalty


    # DHA分类器的损失函数
    def dha_loss(self):
        with self.dha_cda.as_default():
            with tf.name_scope(self.model_name):

                with tf.name_scope('loss_pow'):
                    # len(self.split_num) = 3
                    split_list = np.arange(0, len(self.split_num))
                    for i in range(len(self.split_num)):
                        with tf.name_scope('loss_pow{}'.format(i)):
                            split_list_ = np.delete(split_list, i)  # 按行删除，删除第i+1个数，因为从0开始
                            if len(split_list_) == 1:
                                U_ = self.names['U{}'.format(split_list_[0])]
                            else:
                                U_ = tf.concat([self.names['U{}'.format(j)] for j in split_list_], 0)

                            g1n1 = self.g1n_term(self.names['U{}'.format(i)],  # layer：weight * v + bias
                                                 self.names['Cm{}'.format(i)],  # 球心
                                                 self.names['R{}'.format(i)])  # 半径
                            g2n1 = self.g2n_term(U_,
                                                 self.names['Cm{}'.format(i)],
                                                 self.names['R{}'.format(i)])

                            # 如果 半径 Ri 小于 0 的话，就令Rn = Ri, 否则的话，令Rn = 0
                            Rn = tf.where(tf.greater(np.float64(0), self.names['R{}'.format(i)]),
                                          self.names['R{}'.format(i)], 0)
                            # loss = 该类中所有样本的欧式距离 + P * { 本类样本 + 非本类样本 }
                            # P: 每个球体的惩罚系数 = 1
                            self.names['loss{}_pow'.format(i)] = tf.pow(g1n1, 2) + tf.pow(g2n1, 2) + tf.pow(Rn, 2)

                    loss_pow = 0
                    for i in range(len(self.split_num)):
                        loss_pow = loss_pow + self.names['loss{}_pow'.format(i)]

                        # tf.greater(a,b) 功能：通过比较a、b两个值的大小来输出对错。
                        # 比如：当a=4，b=3时，输出结果为：true；当a=2，b=3时，输出结果为：false。

                        # tf.where的作用是根据condition,返回相对应的x或y,返回值是一个tf.bool类型的Tensor
                        # 比如：A =tf.where(False,123,321) 输出：A = Tensor("Select:0", shape=(), dtype=int32)
                        # 然后sess.run(A) 可得 A = 321
                        # sess.run(tf.where(True,123,321)) = 123

                # tolist():转换为list
                # itertools.combinations：
                # 创建一个迭代器，返回iterable中所有长度为r的子序列，返回的子序列中的项按输入iterable中的顺序排序(不带重复)
                with tf.name_scope('lossR2Cm'):
                    combine = list(itertools.combinations(np.arange(0, len(self.split_num)).tolist(), 2))
                    loss_R2Cm = 0
                    for i in combine:
                        with tf.name_scope('lossR2Cm{}{}'.format(i[0], i[1])):
                            # Cm_normal：两个球的半径的和 - 两个球的中心差的线性范数
                            Cm_normal = (self.names['R{}'.format(i[0])] + self.names['R{}'.format(i[1])]) \
                                        - tf.linalg.norm(self.names['Cm{}'.format(i[0])] - self.names['Cm{}'.format(i[1])])
                            loss_R2Cm = loss_R2Cm + tf.where(tf.greater(Cm_normal, 0), Cm_normal, 0)

                with tf.name_scope('loss_class'):
                    loss_class = 0.0
                    for i in range(len(self.split_num)):
                        loss_class = loss_class + tf.linalg.norm(self.names['U{}'.format(i)] - self.names['Cm{}'.format(i)])

                with tf.name_scope('loss_all'):
                    self.loss_all = self.P * (loss_pow) + self.P_R2Cm * loss_R2Cm + self.P_class * loss_class

            return self.loss_all


    def gen_model(self, train_data, train_label):
        with self.dha_cda.as_default():
            with tf.name_scope(self.model_name):

                for class_i in range(self.fault_num):
                    i_d = np.argwhere(train_label == class_i)
                    i_d = i_d[:, 0]
                    self.split_num.append(len(i_d))
                    self.data_space['data{}'.format(class_i)] = train_data[i_d]
                    self.index_space['index{}'.format(class_i)] = i_d  # 二维

                # names的类型是dict：3，即建立一个字典用来存输入的数据 ‘V0’：shape=(?,13)
                with tf.name_scope('InitVariable'):
                    # 动态变量名设置
                    for i in range(len(self.split_num)):
                        # self.split_num是一个列表：元素为每一类的数据个数 ，即  [52, 63, 45]
                        # len(self.split_num)为类别数
                        with tf.name_scope('V{}'.format(i)):  # self.x_w = data.shape[1] 即列数
                            self.names['V{}'.format(i)] = tf.placeholder(
                                tf.float64, shape=[None, train_data.shape[1]], name='Input{}'.format(i))


                    with tf.name_scope('weight'):
                        self.hidden_w1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[0], self.pop[1]]),
                                                        trainable=True), tf.float64, name='hidden1_w')  # float32
                        output_w1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[1], self.pop[0]]),
                                                        trainable=True), tf.float64, name='output1_w')
                        self.hidden_w2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[1], self.pop[2]]),
                                                        trainable=True), tf.float64, name='hidden2_w')  # float32
                        output_w2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[2], self.pop[1]]),
                                                        trainable=True), tf.float64, name='output2_w')
                        self.hidden_w3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[2], self.pop[3]]),
                                                        trainable=True), tf.float64, name='hidden3_w')  # float32
                        out_w3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[3], self.pop[2]]),
                                                     trainable=True), tf.float64, name='out3_w')

                        self.weights = [self.hidden_w1, self.hidden_w2, self.hidden_w3]


                    with tf.name_scope('bias'):
                        self.hidden_b1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[1]]),
                                                        trainable=True), tf.float64, name='hidden1_b')
                        output_b1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[0]]),
                                                        trainable=True), tf.float64, name='output1_b')
                        self.hidden_b2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[2]]),
                                                        trainable=True), tf.float64, name='hidden2_b')
                        output_b2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[1]]),
                                                        trainable=True), tf.float64, name='output2_b')
                        self.hidden_b3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[3]]),
                                                        trainable=True), tf.float64, name='hidden3_b')
                        out_b3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[2]]),
                                                     trainable=True), tf.float64, name='out3_b')


                    # R_constant:初始化半径
                    # 每个类的球的半径
                    with tf.name_scope('R'):
                        for i in range(len(self.split_num)):
                            self.names['R{}'.format(i)] = tf.Variable(initial_value=self.R_constant,
                                                                 dtype=tf.float64, name='R{}'.format(i), trainable=True)

                with tf.name_scope('encoder_layer'):

                    for i in range(len(self.split_num)):

                        hidden1 = tf.nn.sigmoid(tf.add(tf.matmul(self.names['V{}'.format(i)], self.hidden_w1), self.hidden_b1))
                        output_ae1 = tf.nn.sigmoid(tf.add(tf.matmul(hidden1, output_w1), output_b1))
                        hidden2 = tf.nn.sigmoid(tf.add(tf.matmul(hidden1, self.hidden_w2), self.hidden_b2))
                        output_ae2 = tf.nn.sigmoid(tf.add(tf.matmul(hidden2, output_w2), output_b2))
                        self.names['U{}'.format(i)] = tf.nn.sigmoid(tf.add(tf.matmul(hidden2, self.hidden_w3), self.hidden_b3))
                        out_ae3 = tf.nn.sigmoid(tf.add(tf.matmul(self.names['U{}'.format(i)], out_w3), out_b3))

                        # 逐层训练
                        self.loss_cae1, self.optimizer_cae1 = self.encoder_loss(
                            self.names['V{}'.format(i)], output_ae1, self.hidden_w1, hidden1)

                        self.loss_cae2, self.optimizer_cae2 = self.encoder_loss(
                            hidden1, output_ae2, self.hidden_w2, hidden2)

                        self.loss_cae3, self.optimizer_cae3 = self.encoder_loss(
                            hidden2, out_ae3, self.hidden_w3, self.names['U{}'.format(i)])

                    self.activations = [self.names['V{}'.format(i)], hidden1, hidden2, self.names['U{}'.format(i)]]

                # 球心
                # 对每个类的 每个 layer 求均值
                # axis=0，表示对第一维度（行）减少，减少行的方法是对所有列求平均，即在行上压缩减少为一行。
                with tf.name_scope('circle'):
                    for i in range(len(self.split_num)):
                        self.names['Cm{}'.format(i)] = tf.reduce_mean(self.names['U{}'.format(i)], axis=0)


    def fit(self, train_data, train_label, tree_i):
        '''
        :param train_data: 训练数据集
        :param train_label: 训练集的标签，输入的标签应该是一位的，即1,2, 3......
        :return:
        '''
        with self.dha_cda.as_default():
            with tf.name_scope(self.model_name):

                self.gen_model(train_data, train_label)
                dha_loss = self.dha_loss()

                with tf.name_scope('Optimizer'):
                    learning_rate = tf.train.exponential_decay(self.lr, self.train_step, decay_steps=100,
                                                               decay_rate=0.8)
                    train_op = tf.train.AdamOptimizer(learning_rate).minimize(dha_loss)

                for weight in self.weights:
                    tf.add_to_collection('LRP_CDA_weights'+str(tree_i), weight)

                for act in self.activations:
                    tf.add_to_collection('LRP_CDA_activations'+str(tree_i), act)

                saver = tf.train.Saver()
                with tf.Session(graph=self.dha_cda) as sess:
                    sess.run(tf.global_variables_initializer())
                    # writer = tf.summary.FileWriter("demo_class", sess.graph)
                    # writer.close()

                    feed_dicts = {}
                    for i in range(len(self.split_num)):
                        feed_dicts[self.names['V{}'.format(i)]] = self.data_space['data{}'.format(i)]

                    sess.run([self.loss_cae1, self.optimizer_cae1, self.loss_cae2,self.optimizer_cae2,
                              self.loss_cae3, self.optimizer_cae3], feed_dict=feed_dicts)

                    self.loss_list = []
                    self.R_history = []

                    print('Enter train the Space........')
                    t1_ = time.time()
                    feed_batch_dicts = {}
                    for j in range(self.train_step):
                        for batch_i in range(int(np.ceil(len(train_data) / self.batch_size))):
                            for i in range(len(self.split_num)):
                                feed_batch_dicts[self.names['V{}'.format(i)]] = random.choices(
                                    self.data_space['data{}'.format(i)],
                                    k=int(self.batch_size/self.fault_num))
                            _ = sess.run(train_op, feed_dict=feed_batch_dicts)
                            loss = sess.run(self.loss_all, feed_dict=feed_batch_dicts)

                        self.loss_list.append(loss)
                        R = sess.run([self.names['R{}'.format(i)] for i in range(len(self.split_num))])  # 半径
                        self.R_history.append(R)
                        print(" Epoch", j, ": loss : ", loss)


                    t2_ = time.time()
                    print('训练时间：%.2f s'% (t2_ - t1_))

                    self.R_list = []   # 半径
                    self.circle = []    # 球心
                    self.circle = sess.run([self.names['Cm{}'.format(i)] for i in range(self.fault_num)], feed_dict=feed_dicts)
                    self.R_list = sess.run([self.names['R{}'.format(i)] for i in range(self.fault_num)])

                    w1, w2, w3 = sess.run([self.hidden_w1, self.hidden_w2, self.hidden_w3])
                    b1, b2, b3 = sess.run([self.hidden_b1, self.hidden_b2, self.hidden_b3])
                    self.encoder_weight = [w1, w2, w3]
                    self.encoder_bias = [b1, b2, b3]


                    u_list = sess.run([self.names['U{}'.format(i)] for i in range(self.fault_num)], feed_dict=feed_dicts)
                    DHA_pre = np.zeros(shape=[train_label.shape[0], 1])
                    for i in range(len(self.split_num)):   # 每个类的数据
                        var = u_list[i]
                        dis = np.zeros(shape=[var.shape[0], len(self.split_num)])
                        for j in range(len(self.split_num)):  # 和每个球心求距离

                            center = tf.reshape(tf.tile(self.circle[j], [self.split_num[i]]),
                                                [self.split_num[i], len(self.circle[j])])   # tf.tile 对张量进行复制
                            dis_c = sess.run(tf.linalg.norm(var - center, axis=1))
                            dis_c = dis_c / self.R_list[j]
                            dis[:, j] = dis_c

                        tmp_id = self.index_space['index{}'.format(i)]
                        DHA_pre[tmp_id] = np.argmin(dis, axis=1).reshape(-1, 1)

                    DHA_pre = DHA_pre.astype(np.int64)
                    acc = tf.reduce_mean(tf.cast(tf.equal(DHA_pre, train_label), tf.float32))
                    acc = sess.run(acc)

                    # 画r和loss的变化曲线图
                    dir1 = './DHA_CDA_jnu/'
                    if not os.path.exists(dir1):
                        os.mkdir(dir1)
                    picturesdir = './DHA_CDA_jnu/pictures/'
                    if not os.path.exists(picturesdir):
                        os.mkdir(picturesdir)
                    # fig1 = Visualization(train_label, DHA_pre, picturesdir)
                    # fig1.plot_R(self.R_history, self.split_num)
                    # fig1.plot_loss(self.loss_list)
                    saver.save(sess, "./DHA_CDA_jnu/model_" + str(tree_i))  # 用于LRP
                    sess.close()

                    return DHA_pre, self.circle, self.R_list, self.encoder_weight, self.encoder_bias, loss, acc


    # -------------------距离计算区----------------
    # 测试数据集 在空间W中到球心的距离
    def test(self, testdata, test_label, Cm, R, w, b, test=False):

        with tf.Session() as sess:

            hidden1 = tf.nn.sigmoid(tf.add(tf.matmul(testdata, w[0]), b[0]))
            hidden2 = tf.nn.sigmoid(tf.add(tf.matmul(hidden1, w[1]), b[1]))
            space_test = tf.nn.sigmoid(tf.add(tf.matmul(hidden2, w[2]), b[2]))

            m = space_test.shape[0]
            test_pre = np.zeros(shape=[testdata.shape[0], self.fault_num])
            for i in range(len(self.split_num)):

                center = tf.reshape(tf.tile(Cm[i], [m]), [m, len(Cm[i])])  # tf.tile 对张量进行复制
                distance = sess.run(tf.linalg.norm(space_test - center, axis=1))
                distance /= R[i]
                test_pre[:, i] = distance

            test_pre = np.argmin(test_pre, axis=1).astype(np.int64).reshape(-1, 1)
            acc = tf.reduce_mean(tf.cast(tf.equal(test_pre, test_label), tf.float32))
            acc = sess.run(acc)
            print(acc)

            picturesdir = './DHA_CDA_jnu/pictures/'

            acc = accuracy_score(test_label, test_pre)
            print('accuracy：', acc)
            rec_w = recall_score(test_label, test_pre, average='macro')
            print('recall score (macro)：', rec_w)
            f1_w = f1_score(test_label, test_pre, average='macro')
            print('f1_score (macro)：', f1_w)
            precision_w = precision_score(test_label, test_pre, average='macro')
            print('precision_score (macro)：', precision_w)

            if test == True:
                fig2 = Visualization(test_label, test_pre, picturesdir)
                acc = fig2.plot_confusion_matrix('DHA_CDA test HAR Confusion Matrix')  # 画混淆矩阵
                space_test = sess.run(space_test)
                fig2.pca_2D(space_test)
                fig2.tsne_3d(space_test)
                return acc, test_pre, space_test

                # fig2.pca_encoder_2D(space_test)

            # fig2 = Visualization(test_label, test_pre, picturesdir)
            # acc = fig2.plot_confusion_matrix('DHA_CDA test HAR Confusion Matrix')  # 画混淆矩阵

            # sess.close()
            else:
                return acc, test_pre


In [ ]:
class DHA_SDA():
    def __init__(self, unit1, unit2, unit3, model_name='sda_scope', train_step=10, fault_num=10,
                 batch_size=500, lr=0.001, R_constant = 1.0, P=1.0, P_R2Cm=1.0, P_class=0.0001):
        """
        :param DataFram:  数据集
        :param train_step:训练步数
        :param R_constant:初始化半径
        :param lr:        优化器学习率
        :param P:         每个球体的惩罚系数
        :param P_R2Cm:    球体之间的惩罚系数
        :param P_class:   加速收敛系数
        """
        self.train_step = train_step
        self.lr = lr
        self.P = P
        self.P_R2Cm = P_R2Cm
        self.P_class = P_class
        self.R_constant = R_constant

        # 编码器部分
        self.lambda1 = 1e-4  # 权重衰减项
        self.beta = 0.002  # 稀疏惩罚项系数
        self.rho = 0.2  # 稀疏因子
        self.keep_prop = 0.98
        self.batch_size = batch_size
        self.unit1 = unit1
        self.unit2 = unit2
        self.unit3 = unit3
        self.pop = np.array([600, self.unit1, self.unit2, self.unit3, 1])
        self.model_name = model_name
        self.dha_sda = tf.Graph()
        self.fault_num = fault_num

        self.data_space = {}
        self.index_space = {}
        self.activations = []
        self.names = {}
        self.split_num = []  # 每个类别的数据的个数

        # lrp
        self.weights = []
        self.activations = []


    '''
    加入稀疏性约束，rho_hat是输入层所有神经元在隐层第j个神经元上激活值的平均，rho是稀疏因子
    稀疏自编码器代码： 
    https://github.com/summersunshine1/datamining/blob/master/sparseencoder/sparseAutoEncoder.py
    '''
    # 编码器的损失函数
    def sda_loss(self, data, outputdata, hidden, hidden_w, output_w):

        error_loss = tf.reduce_mean(tf.square(outputdata - data))
        tf.add_to_collection("losses", error_loss)
        # 参数L2正则化：在原来的损失函数基础上加上权重参数的平方和： 限制参数过多或者过大，避免模型更复杂，可以降低过拟合
        regularizer = tf.contrib.layers.l2_regularizer(self.lambda1)
        regularization = regularizer(hidden_w) + regularizer(output_w)
        tf.add_to_collection("losses", regularization)
        # 加入稀疏性约束
        rho_hat = tf.reduce_mean(hidden, 0)
        Sparse_cost = self.beta * (tf.reduce_sum(self.rho * tf.log(self.rho / rho_hat) + (1 - self.rho) *
                                                 tf.log((1 - self.rho) / (1 - rho_hat))))
        tf.add_to_collection("losses", Sparse_cost)

        loss_sae = tf.add_n(tf.get_collection("losses"))
        optimizer_sae = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_sae)
        return loss_sae, optimizer_sae


    # 在圆心外惩罚
    def g1n_term(self, var, center, Rm):
        g1n = tf.linalg.norm(var - center, axis=1) - Rm  # 点到球心的距离 - 半径 = 点到球的距离
        g1n_max = tf.clip_by_value(g1n, 0, 1e10)
        penalty = tf.reduce_mean(g1n_max)  # if res>0, penalty = res else penalty = 0
        return penalty

    # 在圆心内惩罚
    def g2n_term(self, var, center, Rm):
        g2n = Rm - tf.linalg.norm(var - center, axis=1)  # 半径 - 点到球心的距离 = 点到球边缘的距离
        g2n_max = tf.clip_by_value(g2n, 0, 1e10)
        penalty = tf.reduce_mean(g2n_max)  # if res>0,penalty = res else penalty = 0
        return penalty


    # DHA分类器的损失函数
    def dha_loss(self):
        with self.dha_sda.as_default():
            with tf.name_scope(self.model_name):
                with tf.name_scope('loss_pow'):
                    split_list = np.arange(0, len(self.split_num))
                    for i in range(len(self.split_num)):
                        with tf.name_scope('loss_pow{}'.format(i)):
                            split_list_ = np.delete(split_list, i)
                            if len(split_list_) == 1:
                                U_ = self.names['U{}'.format(split_list_[0])]
                            else:
                                U_ = tf.concat([self.names['U{}'.format(j)] for j in split_list_], 0)

                            g1n1 = self.g1n_term(self.names['U{}'.format(i)],  # layer：weight * v + bias
                                                 self.names['Cm{}'.format(i)],  # 球心
                                                 self.names['R{}'.format(i)])  # 半径
                            g2n1 = self.g2n_term(U_,
                                                 self.names['Cm{}'.format(i)],
                                                 self.names['R{}'.format(i)])

                            Rn = tf.where(tf.greater(np.float64(0), self.names['R{}'.format(i)]),
                                          self.names['R{}'.format(i)], 0)
                            self.names['loss{}_pow'.format(i)] = tf.pow(g1n1, 2) + tf.pow(g2n1, 2) + tf.pow(Rn, 2)

                    loss_pow = 0
                    for i in range(len(self.split_num)):
                        loss_pow = loss_pow + self.names['loss{}_pow'.format(i)]


                with tf.name_scope('lossR2Cm'):
                    combine = list(itertools.combinations(np.arange(0, len(self.split_num)).tolist(), 2))
                    loss_R2Cm = 0
                    for i in combine:
                        with tf.name_scope('lossR2Cm{}{}'.format(i[0], i[1])):
                            Cm_normal = (self.names['R{}'.format(i[0])] + self.names['R{}'.format(i[1])]) \
                                        - tf.linalg.norm(self.names['Cm{}'.format(i[0])] - self.names['Cm{}'.format(i[1])])
                            loss_R2Cm = loss_R2Cm + tf.where(tf.greater(Cm_normal, 0), Cm_normal, 0)


                with tf.name_scope('loss_class'):
                    loss_class = 0.0
                    for i in range(len(self.split_num)):
                        loss_class = loss_class + tf.linalg.norm(self.names['U{}'.format(i)] - self.names['Cm{}'.format(i)])

                with tf.name_scope('loss_all'):
                    self.loss_all = self.P * (loss_pow) + self.P_R2Cm * loss_R2Cm + self.P_class * loss_class

            return self.loss_all


    def gen_model(self, train_data, train_label):
        with self.dha_sda.as_default():
            with tf.name_scope(self.model_name):

                for class_i in range(self.fault_num):
                    i_d = np.argwhere(train_label == class_i)
                    i_d = i_d[:, 0]
                    self.split_num.append(len(i_d))
                    self.data_space['data{}'.format(class_i)] = train_data[i_d]
                    self.index_space['index{}'.format(class_i)] = i_d  # 二维


                with tf.name_scope('InitVariable'):
                    for i in range(len(self.split_num)):
                        with tf.name_scope('V{}'.format(i)):
                            self.names['V{}'.format(i)] = tf.placeholder(
                                tf.float64, shape=[None, train_data.shape[1]], name='Input{}'.format(i))


                    with tf.name_scope('weight'):
                        self.hidden_w1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[0], self.pop[1]]),
                                                        trainable=True), tf.float64, name='hidden_w1')  # float32
                        output_w1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[1], self.pop[0]]),
                                                        trainable=True), tf.float64, name='output_w_1')
                        self.hidden_w2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[1], self.pop[2]]),
                                                        trainable=True), tf.float64, name='hidden_w2')  # float32
                        output_w2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[2], self.pop[1]]),
                                                        trainable=True), tf.float64, name='output_w_2')
                        self.hidden_w3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[2], self.pop[3]]),
                                                        trainable=True), tf.float64, name='hidden_w3')  # float32
                        output_w3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[3], self.pop[2]]),
                                                     trainable=True), tf.float64, name='out_w_3')

                        self.weights = [self.hidden_w1, self.hidden_w2, self.hidden_w3]


                    with tf.name_scope('bias'):
                        self.hidden_b1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[1]]),
                                                        trainable=True), tf.float64, name='hidden_b1')
                        output_b1 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[0]]),
                                                        trainable=True), tf.float64, name='output_b_1')
                        self.hidden_b2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[2]]),
                                                        trainable=True), tf.float64, name='hidden_b2')
                        output_b2 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[1]]),
                                                        trainable=True), tf.float64, name='output_b_2')
                        self.hidden_b3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[3]]),
                                                        trainable=True), tf.float64, name='hidden_b3')
                        output_b3 = tf.cast(tf.Variable(initial_value=tf.random_normal(shape=[self.pop[4], self.pop[2]]),
                                                     trainable=True), tf.float64, name='out_b_3')


                    # R_constant:初始化半径
                    with tf.name_scope('R'):
                        for i in range(len(self.split_num)):
                            self.names['R{}'.format(i)] = tf.Variable(initial_value=self.R_constant,
                                                                 dtype=tf.float64, name='R{}'.format(i), trainable=True)


                with tf.name_scope('encoder_layer'):

                    for i in range(len(self.split_num)):
                        hidden1 = tf.nn.sigmoid(tf.add(tf.matmul(self.names['V{}'.format(i)], self.hidden_w1), self.hidden_b1))
                        output_ae1 = tf.nn.sigmoid(tf.add(tf.matmul(hidden1, output_w1), output_b1))
                        hidden1 = tf.nn.dropout(hidden1, keep_prob=self.keep_prop)
                        hidden2 = tf.nn.sigmoid(tf.add(tf.matmul(hidden1, self.hidden_w2), self.hidden_b2))
                        output_ae2 = tf.nn.sigmoid(tf.add(tf.matmul(hidden2, output_w2), output_b2))
                        hidden2 = tf.nn.dropout(hidden2, keep_prob=self.keep_prop)
                        hidden3 = tf.nn.sigmoid(tf.add(tf.matmul(hidden2, self.hidden_w3), self.hidden_b3))
                        out_ae3 = tf.nn.sigmoid(tf.add(tf.matmul(hidden3, output_w3), output_b3))
                        self.names['U{}'.format(i)] = tf.nn.dropout(hidden3, keep_prob=self.keep_prop)


                        # 逐层训练
                        self.loss_ae1, self.optimizer_ae1 = self.sda_loss(
                            self.names['V{}'.format(i)], output_ae1, hidden1, self.hidden_w1, output_w1)

                        self.loss_ae2, self.optimizer_ae2 = self.sda_loss(
                                hidden1, output_ae2, hidden2, self.hidden_w2, output_w2)

                        self.loss_ae3, self.optimizer_ae3 = self.sda_loss(
                            hidden2, out_ae3, self.names['U{}'.format(i)], self.hidden_w3, output_w3)

                    self.activations = [self.names['V{}'.format(i)], hidden1, hidden2, self.names['U{}'.format(i)]]


                with tf.name_scope('circle'):
                    for i in range(len(self.split_num)):
                        self.names['Cm{}'.format(i)] = tf.reduce_mean(self.names['U{}'.format(i)], axis=0)



    def fit(self, train_data, train_label, tree_i):
        with self.dha_sda.as_default():
            with tf.name_scope(self.model_name):

                self.gen_model(train_data, train_label)
                cl_loss = self.dha_loss()

                with tf.name_scope('Optimizer'):
                    learning_rate = tf.train.exponential_decay(self.lr, self.train_step, decay_steps=100,
                                                               decay_rate=0.8)
                    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cl_loss)

                for weight in self.weights:
                    tf.add_to_collection('LRP_SDA_weights'+str(tree_i), weight)

                for act in self.activations:
                    tf.add_to_collection('LRP_SDA_activations'+str(tree_i), act)
                # we = tf.get_collection('LRP_SDA_weights')
                # print(we)
                # ac = tf.get_collection('LRP_SDA_activations')
                # print(ac)

                saver = tf.train.Saver()
                with tf.Session(graph=self.dha_sda) as sess:
                    sess.run(tf.global_variables_initializer())
                    # writer = tf.summary.FileWriter("demo_class", sess.graph)
                    # writer.close()

                    feed_dicts = {}
                    for i in range(len(self.split_num)):
                        feed_dicts[self.names['V{}'.format(i)]] = self.data_space['data{}'.format(i)]

                    sess.run([self.loss_ae1, self.optimizer_ae1, self.loss_ae2, self.optimizer_ae2,
                              self.loss_ae3, self.optimizer_ae3], feed_dict=feed_dicts)

                    self.loss_list = []
                    self.R_history = []

                    print('Enter train the Space........')
                    t1_ = time.time()
                    feed_batch_dicts = {}
                    for j in range(self.train_step):
                        for batch_i in range(int(np.ceil(len(train_data) / self.batch_size))):
                            for i in range(len(self.split_num)):
                                feed_batch_dicts[self.names['V{}'.format(i)]] = random.choices(self.data_space['data{}'.format(i)],
                                                                                    k=int(self.batch_size/self.fault_num))
                            _ = sess.run(train_op, feed_dict=feed_batch_dicts)
                            loss = sess.run(self.loss_all, feed_dict=feed_batch_dicts)

                        self.loss_list.append(loss)
                        R = sess.run([self.names['R{}'.format(i)] for i in range(len(self.split_num))])  # 半径
                        self.R_history.append(R)
                        print(" Epoch", j, ": loss : ", loss)



                    t2_ = time.time()
                    print('训练时间：%.2f s'% (t2_ - t1_))

                    self.R_list = []
                    self.circle = []
                    self.circle = sess.run([self.names['Cm{}'.format(i)] for i in range(self.fault_num)], feed_dict=feed_dicts)
                    self.R_list = sess.run([self.names['R{}'.format(i)] for i in range(self.fault_num)])

                    w1, w2, w3 = sess.run([self.hidden_w1, self.hidden_w2, self.hidden_w3])
                    b1, b2, b3 = sess.run([self.hidden_b1, self.hidden_b2, self.hidden_b3])
                    self.encoder_weight = [w1, w2, w3]
                    self.encoder_bias = [b1, b2, b3]


                    u_list = sess.run([self.names['U{}'.format(i)] for i in range(self.fault_num)], feed_dict=feed_dicts)
                    DHA_pre = np.zeros(shape=[train_label.shape[0], 1])
                    for i in range(len(self.split_num)): # 每个类的数据
                        var = u_list[i]
                        dis = np.zeros(shape=[var.shape[0], len(self.split_num)])
                        for j in range(len(self.split_num)):  # 和每个球心求距离

                            center = tf.reshape(tf.tile(self.circle[j], [self.split_num[i]]),
                                                [self.split_num[i], len(self.circle[j])])   # tf.tile 对张量进行复制
                            dis_c = sess.run(tf.linalg.norm(var - center, axis=1))
                            dis_c = dis_c / self.R_list[j]
                            dis[:, j] = dis_c

                        tmp_id = self.index_space['index{}'.format(i)]
                        DHA_pre[tmp_id] = np.argmin(dis, axis=1).reshape(-1, 1)

                    DHA_pre = DHA_pre.astype(np.int64)
                    acc = tf.reduce_mean(tf.cast(tf.equal(DHA_pre, train_label), tf.float32))
                    acc = sess.run(acc)

                    # 画r和loss的变化曲线图
                    dir1 = './DHA_SDA_jnu/'
                    if not os.path.exists(dir1):
                        os.mkdir(dir1)
                    picturesdir = './DHA_SDA_jnu/pictures/'
                    if not os.path.exists(picturesdir):
                        os.mkdir(picturesdir)
                    # fig1 = Visualization(train_label, DHA_pre, picturesdir)
                    # fig1.plot_R(self.R_history, self.split_num)
                    # fig1.plot_loss(self.loss_list)

                    saver.save(sess, "./DHA_SDA_jnu/model_" + str(tree_i))
                    # sess.close()
                    return DHA_pre, self.circle, self.R_list, self.encoder_weight, self.encoder_bias, loss, acc


    # -------------------距离计算区----------------
    # 测试数据集 在空间W中到球心的距离
    def test(self, testdata, label, Cm, R, w, b, test=False):

        with tf.Session() as sess:

            hidden1 = tf.nn.sigmoid(tf.add(tf.matmul(testdata, w[0]), b[0]))
            hidden2 = tf.nn.sigmoid(tf.add(tf.matmul(hidden1, w[1]), b[1]))
            space_test = tf.nn.sigmoid(tf.add(tf.matmul(hidden2, w[2]), b[2]))

            m = space_test.shape[0]
            test_pre = np.zeros(shape=[testdata.shape[0], self.fault_num])
            for i in range(len(self.split_num)):

                center = tf.reshape(tf.tile(Cm[i], [m]), [m, len(Cm[i])])  # tf.tile 对张量进行复制
                distance = sess.run(tf.linalg.norm(space_test - center, axis=1))
                distance /= R[i]
                test_pre[:, i] = distance

            test_pre = np.argmin(test_pre, axis=1).astype(np.int64).reshape(-1, 1)
            acc = tf.reduce_mean(tf.cast(tf.equal(test_pre, label), tf.float32))
            acc = sess.run(acc)

            picturesdir = './DHA_SDA_jnu/pictures/'
            if not os.path.exists(picturesdir):
                os.mkdir(picturesdir)

            acc = accuracy_score(label, test_pre)
            print('accuracy：', acc)
            rec_w = recall_score(label, test_pre, average='macro')
            print('recall score (macro)：', rec_w)
            f1_w = f1_score(label, test_pre, average='macro')
            print('f1_score (macro)：', f1_w)
            precision_w = precision_score(label, test_pre, average='macro',zero_division=1)
            print('precision_score (macro)：', precision_w)

            if test==True:
                evaluation = Visualization(label, test_pre, picturesdir)
                space_test = sess.run(space_test)
                # evaluation.pca_2D(space_test)
                # evaluation.tsne_3d(space_test)
                acc = evaluation.plot_confusion_matrix('DHA_SDA test HAR Confusion Matrix')  # 画混淆矩阵
                return acc, test_pre, space_test

            # evaluation = Visualization(label, test_pre, picturesdir)
            #
            # acc = evaluation.plot_confusion_matrix('DHA_SDA test HAR Confusion Matrix')  # 画混淆矩阵

            # evaluation.plot_roc('DHA_SDA roc')

            # sess.close()
            else:
                return acc,  test_pre

In [ ]:
优化算法

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import heapq
from sklearn.decomposition import PCA
import matplotlib
import copy
import os

In [ ]:
def fit_fun(X, data, data_label, encoder, train_step, fault_num):  # 适应函数

    if encoder == 'sda_para':
        sda = DHA_SDA(X[0], X[1], X[2], lr=X[3], train_step=train_step, fault_num=fault_num)
        _, _, _, _, _, cost, accuracy = sda.fit(data, data_label, -2)
    elif encoder == 'cda_para':
        cda = DHA_CDA(X[0], X[1], X[2], lr=X[3], train_step=train_step, fault_num=fault_num)
        _, _, _, _, _, cost, accuracy = cda.fit(data, data_label, -2)
    elif encoder == 'dda_para':
        dda = DHA_DDA(X[0], X[1], X[2], lr=X[3], train_step=train_step, fault_num=fault_num)
        _, _, _, _, _, cost, accuracy = dda.fit(data, data_label, -2)

    return cost, accuracy


class Particle:  # 粒子
    # 初始化
    def __init__(self, para_max, para_min, dim, data, data_label, encoder_para, fault_num, encoder_iternum=20):

         # 粒子的位置
        self.fitnessValue_cost = None
        self.pos = [(np.random.rand() * (para_max[i] - para_min[i]))+ para_min[i]
                  for i in range(dim)]
        for i in range(len(self.pos)):
            if i != len(self.pos)-1:
                tmp_x = math.modf(self.pos[i])  # 分别取出整数部分和小数部分  输出tuple（1，2）2为整数部分，1为小数部分
                x_1 = int(tmp_x[1])  # 把整数部分转换为int
                if tmp_x[0] >= 0.5:
                    self.pos[i] = x_1 + 1
                else:
                    self.pos[i] = x_1

        self.bestPos = [0.0 for i in range(dim)]  # 粒子最好的位置
        cost, _ = fit_fun(self.pos, data, data_label, encoder_para, encoder_iternum, fault_num)  # 适应度函数值
        self.fitnessValue_cost = cost

    def set_pos(self, value):
        self.pos = value

    def get_pos(self):
        return self.pos

    def set_best_pos(self, value):
        self.bestPos = value

    def get_best_pos(self):
        return self.bestPos

    def set_fitness_value(self, value1):
        self.fitnessValue_cost = value1

    def get_cost_value(self):
        return self.fitnessValue_cost


class AOA:
    def __init__(self, size, aoa_iter_num, data, data_label, sda_para=False, cda_para=False, dda_para=False,
                 best_fitness_cost=float('Inf'),  dim=4, encoder_iternum=10, fault_num=10):

        self.dim = dim  # 粒子的维度
        self.size = size  # 粒子个数
        self.iter_num = aoa_iter_num  # aoa迭代次数
        self.encoder_iternum = encoder_iternum
        self.data = data
        self.data_label = data_label
        self.fault_num = fault_num
        self.u = 0.5
        self.alpha = 5
        self.MOA_min = 0.2
        self.MOA_max = 0.9
        self.best_fitness_cost = best_fitness_cost
        self.fitness_cost_list = []  # 每次迭代最优适应值
        self.position_list = []
        self.best_position = [0.0 for i in range(dim)]  # 种群最优位置


        # self.para_max = [447, 334, 250, 0.01]
        # self.para_min = [120, 75, 50, 0.002]
        self.para_max = []
        self.para_min = []
        input_unit = self.data.shape[1]
        out_unit = 3 * self.fault_num
        r = np.power(input_unit/out_unit, 1/3)
        min_unit1 = out_unit * np.power(r, 2)
        min_unit2 = out_unit * r
        self.para_min = [int(min_unit1), int(min_unit2), int(out_unit), 0.002]
        for i in range(self.dim-1):
            input_unit = np.sqrt(0.55 * np.power(input_unit, 2) + 3.31 * input_unit + 0.35) + 0.51
            self.para_max.append(int(input_unit))
        self.para_max.append(0.09)


        if sda_para == True:
            self.encoder_para = 'sda_para'
        elif cda_para == True:
            self.encoder_para = 'cda_para'
        elif dda_para == True:
            self.encoder_para = 'dda_para'


        # 初始化粒子
        self.Particle_list = []
        for i in range(self.size):
            p = Particle(self.para_max, self.para_min, self.dim, self.data,self.data_label,
                                       self.encoder_para, fault_num =self.fault_num ,encoder_iternum=self.encoder_iternum)
            self.position_list.append(p.pos)
            self.Particle_list.append(p)

        self.Particle_list.sort(key=lambda x:  x.fitnessValue_cost)
        self.best_position = copy.deepcopy(self.Particle_list[0].pos)
        self.best_fitness_cost = self.Particle_list[0].fitnessValue_cost


    def set_bestFitnessValue(self, value1):
        self.best_fitness_cost = value1

    def get_bestCostValue(self):
        return self.best_fitness_cost

    def set_bestPosition(self, value):
        self.best_position = value

    def get_bestPosition(self):
        return self.best_position


    # 更新位置
    def update_pos(self, part, C_iter):

        # 数学优化器概率（mop），此处MOP表示当前迭代时的函数值，α是一个敏感参数，定义了迭代过程中的开发精度，根据本文的实验，该精度固定为5。
        MOP = 1 - ((C_iter) ** (1 / self.alpha) / (self.iter_num) ** (1 / self.alpha))
        # 数学优化器加速（Math Optimizer Accelerated function）函数：MOA，
        # 此处MOA表示当前迭代的函数值，MOA_min，MOA_max表示加速函数的最小值和最大值， C_iter表示当前是第几次迭代
        MOA = self.MOA_min + C_iter * ((self.MOA_max - self.MOA_min) / self.iter_num)

        new_pos = [0] * self.dim
        best_pos = self.get_bestPosition()

        for i in range(self.dim):
            # （r1、r2和r3）生成[0,1]之间的随机值
            r1 = np.random.rand()
            r2 = np.random.rand()
            r3 = np.random.rand()

            UB = self.para_max[i]
            LB = self.para_min[i]

            '''
            AOA的探索算子在几个区域上随机探索搜索区域，并基于两种主要搜索策略（除法（Division）搜索策略和乘法搜索策略）寻找更好的解决方案，
            这两种策略在等式（3）中建模。
            对于r1>MOA（r1是一个随机数）的条件，该搜索阶段由数学优化器加速（MOA）函数进行调节。
            在该阶段（等式（3）中的第一条规则）中，第一个运算符（Division）的条件为r2<0。5，其他运算符（Multiplication）将被忽略，直到该操作员完成其当前任务。
            否则，第二个运算符（Multiplication）将参与执行当前任务，而不是Division（r2是一个随机数）。
            注意，元素考虑了一个随机比例系数，以产生更多的多样化过程并探索搜索空间的不同区域。
            '''
            # 位置更新方程
            if r1 > MOA:  # r1是一个随机数
                # Exploration phase 勘探阶段
                if r2 > 0.5:  # < ?
                    # Division， 应用除法数学运算符
                    new_pos[i] = best_pos[i] / (MOP + math.e) * ((UB - LB) * self.u + LB)
                else:
                    # Multiplication
                    new_pos[i] = best_pos[i] * MOP * ((UB - LB) * self.u + LB)

            else:
                # Exploitation phase
                if r3 > 0.5:  # < ?
                    # Subtraction
                    new_pos[i] = best_pos[i] - MOP * ((UB - LB) * self.u + LB)
                else:
                    # Addiction
                    new_pos[i] = best_pos[i] + MOP * ((UB - LB) * self.u + LB)

            if new_pos[i] < LB:
                new_pos[i] = LB  # 下限
            elif new_pos[i] > UB:
                new_pos[i] = UB  # 限制解的范围，上限

            # 单元数应该为int型，学习率可以是小数
            if 0 <= i < self.dim-1:
                if isinstance(new_pos[i], float):
                    p = math.modf(new_pos[i])  # 分别取出整数部分和小数部分  输出tuple（1，2）2为整数部分，1为小数部分
                    p_1 = int(p[1])  # 把整数部分转换为int
                    if p[0] >= 0.5:
                        new_pos[i] = p_1 + 1
                    else:
                        new_pos[i] = p_1

            # 限制范围
            if new_pos[i] < LB:
                new_pos[i] = LB
            elif new_pos[i] > UB:
                new_pos[i] = UB

        part.set_pos(new_pos)
        self.position_list.append(new_pos)
        cost, _ = fit_fun(part.get_pos(), self.data, self.data_label, self.encoder_para, train_step=self.encoder_iternum)
        if cost < part.get_cost_value():
            part.set_fitness_value(cost)
            part.set_best_pos(part.get_pos())  # 局部最优值

        if cost < self.get_bestCostValue():  # 全局最优值
            self.set_bestFitnessValue(cost)
            self.set_bestPosition(part.get_pos())


    def update(self):

        for i in range(self.iter_num):
            # 优化
            for part in self.Particle_list:
                self.update_pos(part, i)  # 更新位置

            costvalue = self.get_bestCostValue()
            self.fitness_cost_list.append(costvalue)  # 每次迭代完把当前的最优适应度存到列表
            if costvalue <= 1.0:
                break

        fina_pos = self.get_bestPosition()
        dir1 = './DHA_Bagging/AOA_out_para/'
        if not os.path.exists(dir1):
            os.mkdir(dir1)
        f = open('./DHA_Bagging/AOA_out_para/' + self.encoder_para + '.txt', 'w')
        print(fina_pos, file=f)
        f.close()

        print("PSO最优位置:" + str(self.get_bestPosition()))
        print("PSO cost最优解:" + str(self.fitness_cost_list))
        self.plot(self.fitness_cost_list, self.iter_num)
        self.plot_pos(self.position_list)

        return self.get_bestPosition()

    def plot(self, results, num):
        '''画图
        '''
        X = []
        Y = []
        for i in range(num):
            X.append(i + 1)
            Y.append(results[i])
        plt.plot(X, Y)
        plt.xlabel('Number of iteration', size=10)
        plt.ylabel('Value of cost', size=10)
        plt.title('encoder parameter optimization')
        plt.show()


    def plot_pos(self, pos_list):
        pos_list = np.array(pos_list)
        pca = PCA(n_components=2)
        pca_pos = pca.fit_transform(pos_list)

        plt.figure()
        for i in range(len(pos_list)):
            plt.scatter(pca_pos[i, 0], pca_pos[i, 1], c=i//self.size, marker='o')

        plt.legend()
        plt.show()


In [ ]:
class Bagging_encoder(object):
    # __slots__ = ["forest", "para_forest", "n_estimators", "subsamples", "subfeatures", 'probablity_train',
    # 'train_step', 'aoa_size', 'aoa_iter_num', 'fault_num']
    def __init__(self, n_estimators, subsamples, subfeatures, train_step,
                 aoa_size=4, aoa_iter_num=20, fault_num=10):
        '''
        :param n_estimators: 分类器的总数量
        :param subsamples: 子采样个数
        :param subfeatures: 随机选取特征个数
        :param train_step: 分类器的迭代次数
        :param pso_size: aoa的粒子数目
        :param pso_iter_num: aoa训练时的迭代次数
        '''

        self.n_estimators = n_estimators
        self.subsamples = subsamples
        self.subfeatures = subfeatures
        self.fault_num = fault_num
        self.forest = list()
        self.probablity_train = np.ones(shape=(self.n_estimators, self.fault_num, self.fault_num))  # 训练集条件概率
        self.aoa_size = aoa_size  # 粒子数目
        self.aoa_iter_num = aoa_iter_num   # 迭代次数
        self.para_forest = []
        self.train_step = train_step

    def __chooseSubset(self, X, y):
        '''
        :param X: 输入训练特征集合
        :param y: 输入标签值集合
        :return: 训练子集
        '''
        N, cols = X.shape
        # 随机选取特征 , random.sample从list（即range(0, cols)）中随机选取self.subfeatures个元素，作为一个列表
        # select_cols = list(random.sample(range(0, cols), self.subfeatures))
        # 随机选取训练子集， np.random.permutation(N)对0-N之间的序列随机排序
        select_rows = list(np.random.permutation(N)[:self.subsamples])
        # 获取训练子集
        X_subset = X[select_rows, :]
        # X_subset = X_subset[:, select_cols]
        y_subset = y[select_rows]
        return X_subset, y_subset

    def select_encoder(self, train_data, train_label, n_folds=3):

        coeff = 2
        kf = KFold(n_splits=n_folds)
        weight_ratio = [1 / 3, 1 / 3, 1 / 3]

        for ii, (train_index, verify_index) in enumerate(kf.split(train_data)):
            print('第', ii+1, '折交叉验证')

            x_train, y_train = train_data[train_index], train_label[train_index]
            verify_x, verify_y = train_data[verify_index], train_label[verify_index]

            dha_sda = DHA_SDA(self.pos['sda'][0], self.pos['sda'][1], self.pos['sda'][2], lr=self.pos['sda'][3],
                              train_step=self.train_step, fault_num=self.fault_num)
            dha_dda = DHA_DDA(self.pos['dda'][0], self.pos['dda'][1], self.pos['dda'][2], lr=self.pos['dda'][3],
                              train_step=self.train_step, fault_num=self.fault_num)
            dha_cda = DHA_CDA(self.pos['cda'][0], self.pos['cda'][1], self.pos['cda'][2], lr=self.pos['cda'][3],
                              train_step=self.train_step, fault_num=self.fault_num)

            sda_pre, sda_circle, sda_R, sda_w, sda_b, _, _ = dha_sda.fit(x_train, y_train, -1)
            acc_sda, _ = dha_sda.test(verify_x, verify_y, sda_circle, sda_R, sda_w, sda_b)

            cda_pre, cda_circle, cda_R, cda_w, cda_b, _, _ = dha_cda.fit(x_train, y_train, -1)
            acc_cda, _ = dha_cda.test(verify_x, verify_y, cda_circle, cda_R, cda_w, cda_b)

            dda_pre, dda_circle, dda_R, dda_w, dda_b, _, _ = dha_dda.fit(x_train, y_train, -1)
            acc_dda, _ = dha_dda.test(verify_x, verify_y, dda_circle, dda_R, dda_w, dda_b)

            acc_verify = [acc_sda, acc_dda, acc_cda]
            alpha = [np.exp(coeff * h) for h in acc_verify]

            sum_w_1 = alpha[0] * weight_ratio[0]
            sum_w_2 = alpha[1] * weight_ratio[1]
            sum_w_3 = alpha[2] * weight_ratio[2]
            sum = sum_w_1 + sum_w_2 + sum_w_3

            for j in range(3):
                weight_ratio[j] = (weight_ratio[j] * alpha[j]) / sum
            print(weight_ratio)

        return weight_ratio

    def fit(self, X, y):
        '''
        :param X: 输入训练特征集合
        :param y: 输入标签值集合，一维
        :return: 随机森林回归器
        '''
        start_train = time.clock()
        self.pos = {}

        # aoa调参
        aoa_sda = AOA(self.aoa_size, self.aoa_iter_num, X, y, sda_para=True,
                      encoder_iternum=self.train_step, fault_num=self.fault_num)
        best_pos_sda = aoa_sda.update()
        print("DHA_SDA最优位置:" + str(best_pos_sda))
        self.pos['sda'] = best_pos_sda

        aoa_dda = AOA(self.aoa_size, self.aoa_iter_num, X, y, dda_para=True,
                      encoder_iternum=self.train_step, fault_num=self.fault_num)
        best_pos_dda = aoa_dda.update()
        print("DHA_DDA最优位置:" + str(best_pos_dda))
        self.pos['dda'] = best_pos_dda

        aoa_cda = AOA(self.aoa_size, self.aoa_iter_num, X, y, cda_para=True,
                      encoder_iternum=self.train_step, fault_num=self.fault_num)
        best_pos_cda = aoa_cda.update()
        print("DHA_CDA最优位置:" + str(best_pos_cda))
        self.pos['cda'] = best_pos_cda

        for i in range(self.n_estimators):
            # 获取构建单棵树的训练子集
            X_subset, y_subset = self.__chooseSubset(X, y) # y_subset：一维
            # 训练单棵树
            print('选择第', i+1, '个树的分类器形式')
            w, pos = self.select_encoder(X_subset, y_subset)   # pos: sda dda cda

            if max(w) == w[0]:
                onetree = DHA_SDA(pos[0][0], pos[0][1], pos[0][2], lr=pos[0][3],
                                  train_step=self.train_step, fault_num=self.fault_num)
                print('第', i+1, '个分类器为DHA_SDA')
                sda_pre, sda_circle, sda_R, sda_w, sda_b, _, _ = onetree.fit(X_subset, y_subset, i)
                cm_estimators = confusion_matrix(y_subset, sda_pre)  # 混淆矩阵
                para = [sda_circle, sda_R, sda_w, sda_b]

            elif max(w) == w[1]:
                onetree = DHA_DDA(pos[1][0], pos[1][1], pos[1][2], lr=pos[1][3],
                                  train_step=self.train_step, fault_num=self.fault_num)
                print('第', i+1, '个分类器为DHA_DDA')
                dda_pre, dda_circle, dda_R, dda_w, dda_b, _, _ = onetree.fit(X_subset, y_subset, i)
                cm_estimators = confusion_matrix(y_subset, dda_pre)  # 混淆矩阵
                para = [dda_circle, dda_R, dda_w, dda_b]

            else:
                onetree = DHA_CDA(pos[2][0], pos[2][1], pos[2][2], lr=pos[2][3],
                                  train_step=self.train_step, fault_num=self.fault_num)
                print('第', i+1, '个编码器为DHA_CDA')
                cda_pre, cda_circle, cda_R, cda_w, cda_b, _, _ = onetree.fit(X_subset, y_subset, i)
                cm_estimators = confusion_matrix(y_subset, cda_pre)  # 混淆矩阵
                para = [cda_circle, cda_R, cda_w, cda_b]


            # 组成随机森林
            self.forest.append(onetree)
            self.para_forest.append(para)
            for i in range(len(self.forest)):
                print('第' + str(i) + '个分类器是：')
                print(self.forest[i])
                print('参数是：')
                print(self.para_forest[i])

            cm_estimators = cm_estimators.astype(np.float64)
            for j in range(self.fault_num):  # 真实标签为j  :行
                m = len(np.argwhere(y_subset == j).flatten())        # 每种标签的数量
                label_pre = (m + 1) / (y_subset.shape[0] + y_subset.shape[1])  # 每种标签k的先验  laplace校准
                for jj in range(self.fault_num):  # 预测标签为jj  :列
                    cm_estimators[j][jj] = cm_estimators[j][jj] / m  # 混淆矩阵的概率形式 0.01防止分母为0
                    self.probablity_train[i, j, jj] = label_pre * cm_estimators[j][jj]  # 后验概率

        print(self.forest)
        end_train = time.clock()
        print(' train time: %s Seconds' % (end_train - start_train))

    def predict(self, X, y):

        start_test = time.clock()
        assert len(X) > 0

        # predict = np.zeros(shape=(X.shape[0], self.n_estimators))  # q个分类器的输出  E（m,q）
        predict = []
        final_pre = np.zeros(shape=(X.shape[0], 1))   # 最终的预测

        for i in range(self.n_estimators):
            _, y_pre = self.forest[i].test(X, y,
                self.para_forest[i][0], self.para_forest[i][1], self.para_forest[i][2], self.para_forest[i][3])    # y_pre：shape：（，1）

            predict.append(y_pre)


        index_lrp = []
        index = 0
        for m in range(X.shape[0]):
            max = 0
            axis_x = y[m][0]  # 实际标签
            for j in range(self.n_estimators):
                axis_y = int(predict[j][m])  # 第j个编码器对第i个样本的预测标签
                t = self.probablity_train[j][axis_x][axis_y]  # 第j个编码器的 后验概率
                if t > max:
                    max = t
                    final_pre[m] = predict[j][m]
                    index = j      # 使用的哪个分类器的分类结果
            index_lrp.append(index)

        final_pre = final_pre.astype(np.int64)


        dir1 = './DHA_AOA_Bagging/'
        if not os.path.exists(dir1):
            os.mkdir(dir1)
        picturesdir = './DHA_AOA_Bagging/bagging_result_pictures/'
        if not os.path.exists(picturesdir):
            os.mkdir(picturesdir)

        fig = Visualization(y, final_pre, picturesdir)
        acc = fig.plot_confusion_matrix('DHA_AOA_bagging test HAR Confusion Matrix' )  # 画混淆矩阵
        # fig.tsne_precl(X, 'DHA_AOA_bagging Initial data')
        # fig.tsne_af(X,  'DHA_AOA_bagging Data after classification')
        fig.plot_roc('DHA_AOA_bagging ROC curve')  # 画roc图
        end_test = time.clock()
        print('test time: %s Seconds' % (end_test - start_test))

        return final_pre, index_lrp

In [ ]:
data = shujuchuli.cut_samples(0)
train_x, train_y, test_x, test_y = shujuchuli.make_datasets(data)


suiji_encoder = Bagging_encoder(n_estimators=3,
                                subsamples=2*train_x.shape[0]//3,
                                subfeatures=train_x.shape[1]//2,
                                train_step=300,
                                aoa_size=2, aoa_iter_num=20)
encoder_li = suiji_encoder.fit(train_x, train_y)

# 模型验证

rf_test_pre, index_lrp = suiji_encoder.predict(test_x, test_y)

